#### R-tree Spatial Indexing with Python

Practicamos la indexación espacial de  [Geoff Boeing](https://geoffboeing.com/2016/10/r-tree-spatial-index-python/) usando GeoPandas. En este caso vamos a usar los geodataframes que venimos usando en este archivo.

In [3]:
import matplotlib.pyplot as plt
import h3
import pandas as pd 
import geopandas as gpd
import folium
import branca.colormap as cm

ModuleNotFoundError: No module named 'h3'

In [4]:
# cargar los geodataframe desde su directorio
barrios = gpd.read_file(r"C:\Users\UserMCR\Desktop\Agustín\censo2022\Proyecto\GeoPandasOptimizado\5344\barrios.shp")
radios = gpd.read_file(r"C:\Users\UserMCR\Desktop\Agustín\censo2022\Proyecto\GeoPandasOptimizado\5344\radios.shp")
union_2 = gpd.read_file(r"C:\Users\UserMCR\Desktop\Agustín\censo2022\Proyecto\proyecto_avanzado\union2\union2.shp")

NameError: name 'gpd' is not defined